In [2]:
import matplotlib.pyplot as plt
import numpy as np
from typing import Callable

In [3]:
# system variables
rho = 1
length = 1
u_psi = 5
phi_max = np.pi/6

r_min = length / (np.tan(phi_max))

In [4]:
def nonlinearCar(q: np.ndarray, u: np.ndarray, t: float):
    return np.array([
        rho * np.cos(q[2]),
        rho * np.sin(q[2]),
        rho / length * np.tan(q[4]),
        u[0],
        u[1],
        u[2]
    ])

def aMat(t: float, qt: np.ndarray, ut: np.ndarray):
    A = np.zeros([6, 6])
    A[0, 2] = - ut[0] * rho * np.sin(qt[2])
    A[1, 2] = ut[0] * rho * np.cos(qt[2])
    A[2, 4] = ut[0] * rho / length * np.cos(qt[4]) ** -2
    return A

def bMat(t: float, qt: np.ndarray, ut: np.ndarray):
    B = np.zeros([6, 3])
    B[0, 0] = rho * np.cos(qt[2])
    B[1, 0] = rho * np.sin(qt[2])
    B[2, 0] = rho / length * np.tan(qt[4])
    B[3, 0] = 1
    B[4, 1] = 1
    B[5, 2] = 1
    return B

def linearizedCar(q: np.ndarray, u: np.ndarray, t: float, q_tilde: 'Callable[[float], np.ndarray]', u_tidle: 'Callable[[float], np.ndarray]'):
    qt = q_tilde(t)
    ut = u_tidle(t)
    f = nonlinearCar(qt, ut, t)
    A = aMat(t, qt, ut)
    B = bMat(t, qt, ut)
    return f + np.matmul(A, q - qt) + np.matmul(B, u - ut)



In [5]:
def straightLineGenerator(x0, y0, theta0, v):
    def f(t):
        return np.ndarray([
            x0 + v * rho * np.cos(theta0) * t,
            x0 + v * rho * np.cos(theta0) * t,
            0,
            v * t,
            0,
            0
        ])

def curvePathGenerator(x0, y0, theta0, theta_dot, phi, v):
    r = length / np.tan(phi)
    if phi > 0: # left
        perp = theta0 + np.pi / 2
    else: # right
        perp = theta0 - np.pi / 2
    xc = r * np.cos(perp) + x0
    yc = r * np.sin(perp) + y0
    def f(t):
        return np.ndarray([
            xc + r * np.cos(theta_dot * t + theta0),
            yc + r * np.sin(theta_dot * t + theta0),
            theta_dot * t,
            v * t,
            phi,
            0
        ])
    return f


$\Phi[k, k_0] = A[k -1]A[k-2]\cdots A[k_0]$


$x[k] = \Phi[k, k_0]x_0 + \sum_{m=k_0}^{k-1}\Phi[k, m + 1]B[m]u[m]$


$k_0 = k-1$


$x[k] = \Phi[k, k-1]x_{k-1} + \Phi[k, k-1]B[k-1]u[k-1]$


$x[k] = A[k-1]x_{k-1} + A[k-1]B[k-1]u[k-1]$

In [6]:
h = .1

def straightAmat(q0):
    return np.array([
        [1, 0, -u_psi * rho * np.sin(q0[2]) * h, 0, 0, 0],
        [0, 1, u_psi * rho * np.cos(q0[2]) * h, 0, 0, 0],
        [0, 0, 1, 0, u_psi * rho / length * h, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1]
    ])

def straightBmat(q0):
    return np.array([
        [rho * np.cos(q0[2]) * h, 0, 0],
        [rho * np.sin(q0[2]) * h, 0, 0],
        [0, 0, 0],
        [h, 0, 0],
        [0, h, 0],
        [0, 0, h]
    ])

def straightDmat(q0):
    return np.array([
        rho * u_psi * np.cos(q0[2]) * h,
        rho * u_psi * np.sin(q0[2]) * h,
        0,
        u_psi * h,
        0,
        0
    ])

In [9]:
from sympy import Symbol, sin, cos, sec, tan, Matrix, integrate, eye, latex
u_psi = Symbol('u_psi')
h = Symbol('h')
rho = Symbol('rho')
t = Symbol('t')
theta = Symbol('theta')
l = Symbol('l')
phi = Symbol('phi')

A = Matrix([
    [ 0, 0, -u_psi * rho * sin(theta), 0, 0, 0],
    [ 0, 0, u_psi * rho * cos(theta), 0, 0, 0],
    [ 0, 0, 0, 0,u_psi * rho / l * sec(phi) ** 2 , 0],
    [ 0, 0, 0, 0, 0, 0],
    [ 0, 0, 0, 0, 0, 0],
    [ 0, 0, 0, 0, 0, 0],
])
B = Matrix([
    [rho * cos(theta), 0, 0],
    [rho * sin(theta), 0, 0],
    [rho / l * tan(phi), 0, 0],
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])
A * A * A

Matrix([
[0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0]])

In [13]:
u_psi = Symbol('u_psi')
h = Symbol('h')
rho = Symbol('rho')
t = Symbol('t')
k = Symbol('k')
t_0 = Symbol('t_0')
theta_0 = Symbol('theta_0')
theta_dot = Symbol('thetadot', nonzero=True)
l = Symbol('l')
phi_c = Symbol('phi_c')

A = Matrix([
    [ 0, 0, -u_psi * rho * sin(theta_dot * t + theta_0), 0, 0, 0],
    [ 0, 0, u_psi * rho * cos(theta_dot * t + theta_0), 0, 0, 0],
    [ 0, 0, 0, 0,u_psi * rho / l * sec(phi_c) ** 2 , 0],
    [ 0, 0, 0, 0, 0, 0],
    [ 0, 0, 0, 0, 0, 0],
    [ 0, 0, 0, 0, 0, 0],
])
B = Matrix([
    [rho * cos(theta_dot * t + theta_0), 0, 0],
    [rho * sin(theta_dot * t + theta_0), 0, 0],
    [rho / l * tan(phi_c), 0, 0],
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

D = Matrix([
    rho * u_psi * cos(theta_dot * t + theta_0),
    rho * u_psi * sin(theta_dot * t + theta_0),
    rho / l * u_psi * tan(phi_c),
    u_psi,
    0,
    0
])
t_0 = 0
    

def Phi(r, tau):
    return eye(6) + integrate(A, (t, tau, r)) + integrate(A * integrate(A, (t, tau, t)), (t, tau , r))

F = Phi(h * k + t_0, t_0)
F
s = Symbol('s')
B = B.subs(t, s)
G = integrate(Phi(h * k + t_0, s) * B, (s, t_0, h * k + t_0))
G.simplify()
# print(latex(F.simplify()))
D = integrate(Phi(h * k + t_0, s) * D, (s, t_0, h * k + t_0))
D.simplify()

Matrix([
[rho*u_psi*(h*k*l*thetadot**2*cos(t*thetadot + theta_0) + rho*u_psi*(h*k*thetadot*cos(h*k*thetadot + theta_0) - sin(h*k*thetadot + theta_0))*tan(phi_c) + rho*u_psi*sin(theta_0)*tan(phi_c))/(l*thetadot**2)],
[rho*u_psi*(h*k*l*thetadot**2*sin(t*thetadot + theta_0) + rho*u_psi*(h*k*thetadot*sin(h*k*thetadot + theta_0) + cos(h*k*thetadot + theta_0))*tan(phi_c) - rho*u_psi*cos(theta_0)*tan(phi_c))/(l*thetadot**2)],
[                                                                                                                                                                                 h*k*rho*u_psi*tan(phi_c)/l],
[                                                                                                                                                                                                  h*k*u_psi],
[                                                                                                                                                                  